In [1]:
import pandas as pd
from snv2 import *
import ray
ray.shutdown()
ray.init()
_output_root = 'multi_clipp_simulation_data_March4/'
df = pd.read_csv(_output_root + 'simulation_data_cluster_5_region_3_read_depth_100_replica_2.tsv', sep = '\t')
n = 20
m = 3

[df.cluster[m * i] for i in range(n)]

2024-10-23 16:20:57,854	INFO worker.py:1786 -- Started a local Ray instance.


[0, 0, 3, 3, 2, 3, 2, 3, 3, 0, 2, 3, 3, 1, 0, 1, 0, 3, 2, 3]

In [3]:
df = df.iloc[0: (n * m), :]


rho = 0.05
gamma = [0.2 * i for i in range(10)]
omega = 1
max_iteration = 50

res = [ADMM2.remote(df, rho, gamma[i], omega, n, m, max_iteration)[3] for i in range(10)]
res = ray.get(res)
ray.shutdown()
res


2024-10-23 16:26:40,162	INFO worker.py:1786 -- Started a local Ray instance.


(ADMM2 pid=44695) Gamma: 0.2, clusters : dict_values([0, 1, 2, 3, 4, 5, 6, 7, 7, 9, 10, 11, 12, 13, 14, 15, 16, 5, 18, 7])
(ADMM2 pid=44697) Gamma: 0.0, clusters : dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
(ADMM2 pid=44699) Gamma: 0.4, clusters : dict_values([0, 1, 2, 3, 4, 5, 6, 7, 7, 9, 10, 11, 12, 13, 14, 15, 15, 5, 18, 12])
(ADMM2 pid=44693) Gamma: 0.6000000000000001, clusters : dict_values([0, 1, 2, 3, 4, 5, 6, 7, 7, 0, 10, 11, 12, 13, 14, 15, 15, 5, 18, 5])
(ADMM2 pid=44694) Gamma: 0.8, clusters : dict_values([0, 1, 2, 3, 4, 5, 6, 5, 5, 0, 10, 11, 5, 13, 1, 1, 1, 5, 18, 5])
(ADMM2 pid=44698) Gamma: 1.0, clusters : dict_values([0, 1, 2, 3, 1, 5, 6, 5, 5, 0, 10, 11, 11, 0, 1, 1, 1, 11, 0, 11])
(ADMM2 pid=44700) Gamma: 1.2000000000000002, clusters : dict_values([0, 1, 2, 3, 1, 3, 1, 3, 3, 0, 1, 3, 3, 0, 1, 1, 1, 3, 0, 3])
(ADMM2 pid=44691) Gamma: 1.4000000000000001, clusters : dict_values([0, 1, 2, 3, 1, 3, 1, 3, 3, 0, 0, 3, 3, 0, 1, 1, 1, 3

[[5089.941078262395, array(-2455.09857092), 0.0],
 [4958.784311200735, array(-2403.00098262), 0.2],
 [4994.1646784413415, array(-2425.18476466), 0.4],
 [5039.233945158026, array(-2452.21299642), 0.6000000000000001],
 [5066.064494409487, array(-2483.60266469), 0.8],
 [5105.527562855863, array(-2516.81499415), 1.0],
 [5145.460691003869, array(-2554.75595186), 1.2000000000000002],
 [5218.077010293847, array(-2591.06411151), 1.4000000000000001],
 [5267.739391272596, array(-2624.88249882), 1.6],
 [5326.409260590397, array(-2654.21743347), 1.8]]

In [1]:
import pandas as pd
from snv import *
df = pd.read_csv('simulation_data_cluster_5_region_6_read_depth_100_replica_2.tsv', sep = '\t')
snv_s = []
for i in range(10):
    start = i * 6
    end = i * 6 + 6
    mat = df.iloc[start : end, ::]
    snv = SNV()
    snv.initialize_by_matrix(mat)
    snv.get_likelihood()
    snv_s.append(snv)
snvs1 = snvs()
snvs1.initialize_by_list(snv_s)
snvs1.set_omega(150)

In [17]:
import torch
X = torch.tensor(2.)
Y = torch.tensor(10.)

class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    init_scaling_factor = torch.zeros(1)
    self.scaling_factor = torch.nn.Parameter(init_scaling_factor, requires_grad=True)
 
  def forward(self, x):
    output = self.scaling_factor * x
    return output

model = Model()
optim = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

n_iters = 8000
for i in range(0, n_iters):
  predictions = model.forward(X)
  loss = loss_fn(predictions, Y)
  loss.backward()
  optim.step()
  optim.zero_grad()
  print(f"loss ({i}): {loss.item()}")
print(f"The scaling parameter is {model.scaling_factor.data.item()}")

/Users/yuding/miniforge3/envs/ml1/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss (0): 100.0
loss (1): 84.63999938964844
loss (2): 71.63928985595703
loss (3): 60.6355094909668
loss (4): 51.32189178466797
loss (5): 43.43885040283203
loss (6): 36.766639709472656
loss (7): 31.119287490844727
loss (8): 26.339366912841797
loss (9): 22.293642044067383
loss (10): 18.869340896606445
loss (11): 15.971010208129883
loss (12): 13.517864227294922
loss (13): 11.441520690917969
loss (14): 9.684102058410645
loss (15): 8.196623802185059
loss (16): 6.937623500823975
loss (17): 5.872004985809326
loss (18): 4.970065593719482
loss (19): 4.206664085388184
loss (20): 3.560520887374878
loss (21): 3.0136234760284424
loss (22): 2.550732374191284
loss (23): 2.1589386463165283
loss (24): 1.8273245096206665
loss (25): 1.5466474294662476
loss (26): 1.309083104133606
loss (27): 1.1080087423324585
loss (28): 0.9378188848495483
loss (29): 0.7937701940536499
loss (30): 0.6718475222587585
loss (31): 0.5686522126197815
loss (32): 0.481306791305542
loss (33): 0.40737810730934143
loss (34): 0.34480

In [40]:
minimized_params

tensor([-300.,   -1.], requires_grad=True)

In [37]:
list_of_params

[tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-300.,   -1.], requires_grad=True),
 tensor([-